# SolvationBuilder

In [ ]:
import veloxchem as vlx

Create a molecule object for deprotonated ibuprofen and generate a forcefield with semiempirical partial charges to use in the following solvationfep calculations. 

In [ ]:
ibuprof = vlx.Molecule.read_smiles('CC(C)CC1=CC=C(C=C1)C(C)C(=O)[O-]')
ibuprof.show()

In [ ]:
ff_gen_solute = vlx.MMForceFieldGenerator()
ff_gen_solute.partial_charges = ibuprof.get_partial_charges(
    ibuprof.get_charge())
ff_gen_solute.create_topology(ibuprof)
ff_gen_solute.write_gromacs_files('ibuprof','MOL')

Load the SolvationBuilder and create a box of padding 1.0 nm (default). Solvate the molecule with water (SPC/E water model). After solvating the box, the SolvationBuilder will run a NPT equilibration of 5 ps at 300K. Once the equilibration is finished, gromacs files for the equilibrated system is written.

In [ ]:
solvator = vlx.SolvationBuilder()

solvator.solvate(ibuprof, equilibrate=True, neutralize=False)
solvator.system_molecule.show()

solvator.write_gromacs_files(solute_ff=ff_gen_solute)

# Free Energy of Solvation

The free energy of solvation for ibuprofen in a mix of water and propylene glycol is computed with the SolvationFepDriver using the gromacs files generated from the SolvationBuilder as input. The functon requires a gro and topology file for both the solvated system and the molecule in vacuum.

In [ ]:
solvationfep = vlx.SolvationFepDriver()

# Optional, saving trajectory from each lambda simulation in xtc format 
#solvationfep.save_trajectory_xtc = True 

# The number of steps here has been chosen for a quick execution in the 
# #Notebook, we recommend using 500 000 steps for production runs (1 ns) per Lambda.
solvationfep.num_steps = 10000
delta_f, final_energy = solvationfep.compute_solvation_from_gromacs_files(
    'system.gro','system.top','ibuprof.gro','ibuprof.top')

## Custom solvate
Following is an example of how to solvate ibuprofen in a mixed solvent of equal amounts of water and propylene glycol. A VeloxChem molecule object is created for solvent molecules. A list of the solvent molecules, the quantity of each molecule and the box dimension (Å) is specified in the function. Gromacs files for the system is written.

In [ ]:
solvator = vlx.SolvationBuilder()

propylene_glycol = vlx.Molecule.read_smiles('CC(CO)O')
water = vlx.Molecule.read_smiles('O')

solvator.custom_solvate(ibuprof,
                        solvents=[water, propylene_glycol],
                        quantities=[200,200],
                        box=(40,40,40))
solvator.system_molecule.show()
solvator.write_gromacs_files(solute_ff=ff_gen_solute)

____
## Free Energy of Solvation for a dataset


In this example the free energy of solvation is calculated for a dataset of molecules. The necessary information about each molecule, in this case the SMILES code and charge, is loaded from the *solvationfep_molecules.csv* file. <n>

Iterate over the molecules to run the solvationfep calculations. Results of the final energy will be updated in the .csv file

The following part of the Notebook is rather time consuming, we recommend to perform those calculations on a cluster instead.

In [ ]:
import veloxchem as vlx
import pandas as pd

solvationfep = vlx.SolvationFepDriver()
solvationfep.padding = 1.0
filename = 'data/solvationfep_molecules.csv'

df = pd.read_csv(filename, sep=',')

results = []

for index, row in df.iterrows():

    print(f'Calculating Free Energy of Solvation for molecule: {row['SMILES']}')
    molecule = vlx.Molecule.read_smiles(row['SMILES'])
    
    delta_f, final_energy = solvationfep.compute_solvation(molecule)
    results.append(f"{final_energy:.4f}")

df['Result kJ/mol'] = results
df.to_csv(filename, sep=',', index=False)

Plot the results for all molecules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('solvationfep_molecules.csv', sep=',')

# Plot the results against SMILES
plt.figure(figsize=(10, 6))
plt.plot(df['SMILES'], df['Result kJ/mol'],'o', color='darkcyan', alpha=0.7)

plt.ylabel(r'$\Delta G^\mathrm{solv}$ Computed (kJ/mol)', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()

plt.show()
